In [1]:
import sys

sys.path.insert(0, '..')

#!{sys.executable} -m pip install folium
#!{sys.executable} -m pip install openapi_core


In [2]:
from morpheus.project.types.discretization.spatial import Rotation, LengthUnit, Grid
from morpheus.project.types.geometry import Polygon

polygon = Polygon(
  type='Polygon',
  coordinates=[[
    (13.922514437551428, 50.964720483303836),
    (13.925250781947113, 50.965228748412386),
    (13.925036413951403, 50.96623732041704),
    (13.92222441026388, 50.96609040370362),
    (13.922514437551428, 50.964720483303836)
  ]]
)

rotation = Rotation(20)
length_unit = LengthUnit.meters()
relative_col_coordinates = [0, 0.1, 0.21, 0.22, 0.23, 0.24, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
relative_row_coordinates = [0, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 1]

grid = Grid.from_polygon_with_relative_coordinates(
  polygon=polygon,
  relative_col_coordinates=relative_col_coordinates,
  relative_row_coordinates=relative_row_coordinates,
  rotation=rotation,
)
polygon

Polygon(coordinates=[[(13.922514437551428, 50.964720483303836), (13.925250781947113, 50.965228748412386), (13.925036413951403, 50.96623732041704), (13.92222441026388, 50.96609040370362), (13.922514437551428, 50.964720483303836)]], type='Polygon')

In [3]:
grid.get_wgs_cell_center_coordinates()

([[13.92201083954258,
   13.922325948095247,
   13.922506010125344,
   13.922536020463694,
   13.922566030802043,
   13.922596041140393,
   13.922686072155441,
   13.92291114969306,
   13.923211253076555,
   13.923511356460047,
   13.923811459843542,
   13.924111563227036,
   13.924411666610526,
   13.92471176999402],
  [13.922094182248257,
   13.92240929080093,
   13.922589352831025,
   13.922619363169375,
   13.922649373507722,
   13.922679383846072,
   13.922769414861119,
   13.922994492398743,
   13.923294595782234,
   13.923594699165726,
   13.923894802549222,
   13.924194905932714,
   13.924495009316207,
   13.924795112699702],
  [13.9221021196488,
   13.92241722820147,
   13.922597290231565,
   13.922627300569914,
   13.922657310908264,
   13.922687321246613,
   13.922777352261662,
   13.923002429799283,
   13.923302533182776,
   13.923602636566269,
   13.923902739949764,
   13.924202843333257,
   13.924502946716748,
   13.924803050100245],
  [13.92211005704934,
   13.9224251656

In [4]:
grid.get_wgs_bbox()

(13.921781413845427, 50.96461542216845, 13.925576187734443, 50.96667675655004)

In [5]:
import folium

m = folium.Map(crs="EPSG3857", zoom_start=12, location=[50.965, 13.922])
m.add_child(folium.GeoJson(polygon.as_geojson()))
m.fit_bounds(m.get_bounds())
m

In [6]:
m.add_child(folium.GeoJson(grid.get_wgs_outline_geometry().as_geojson()))
m.fit_bounds(m.get_bounds())
m

In [7]:

cell_geometries = grid.get_wgs_cell_geometries()

n_x = grid.n_cols()
n_y = grid.n_rows()

for col in range(n_x):
  for row in range(n_y):
    cell_geometry = cell_geometries[row][col]
    folium.GeoJson(cell_geometry.as_geojson()).add_to(m)

m

In [8]:
# Render GRid Rows and Columns
from morpheus.project.types.discretization.spatial import ActiveCells

cells = ActiveCells.from_polygon(polygon, grid)

m = folium.Map(zoom_start=12, location=[50.965, 13.922])
folium.GeoJson(polygon.as_geojson(), name="hello world").add_to(m)
folium.GeoJson(grid.to_geojson().as_geojson()).add_to(m)
m.fit_bounds(m.get_bounds())
m


[(1, 0)] [(8, 0)]
[(1, 1)] [(10, 1)]
[(1, 2)] [(11, 2)]
[(1, 3)] [(11, 3)]
[(1, 4)] [(11, 4)]
[(1, 5)] [(12, 5)]
[(1, 6)] [(12, 6)]
[(0, 7)] [(13, 7)]
[(0, 8)] [(13, 8)]
[(0, 9)] [(13, 9)]
[(0, 10)] [(13, 10)]
[(0, 11)] [(13, 11)]


In [9]:
from morpheus.project.types.discretization.spatial import ActiveCells

cells_geometries = ActiveCells.from_polygon(polygon, grid).to_geojson(grid)

m = folium.Map(zoom_start=12, location=[50.965, 13.922])
folium.GeoJson(polygon.as_geojson(), name="hello world").add_to(m)
folium.GeoJson(grid.get_wgs_outline_geometry().as_geojson()).add_to(m)
folium.GeoJson(cells_geometries.as_geojson()).add_to(m)
m.fit_bounds(m.get_bounds())
m

[(1, 0)] [(8, 0)]
[(1, 1)] [(10, 1)]
[(1, 2)] [(11, 2)]
[(1, 3)] [(11, 3)]
[(1, 4)] [(11, 4)]
[(1, 5)] [(12, 5)]
[(1, 6)] [(12, 6)]
[(0, 7)] [(13, 7)]
[(0, 8)] [(13, 8)]
[(0, 9)] [(13, 9)]
[(0, 10)] [(13, 10)]
[(0, 11)] [(13, 11)]


In [10]:
cells_outline = cells.outline_to_geojson(grid)

m = folium.Map(zoom_start=12, location=[50.965, 13.922])
folium.GeoJson(polygon.as_geojson()).add_to(m)
folium.GeoJson(grid.get_wgs_outline_geometry().as_geojson()).add_to(m)
folium.GeoJson(cells_outline.as_geojson()).add_to(m)
m.fit_bounds(m.get_bounds())

m

In [11]:
cells_outline = cells.outline_to_geojson(grid)
m = folium.Map(zoom_start=12, location=[50.965, 13.922])
folium.GeoJson(cells_outline.as_geojson()).add_to(m)
m.fit_bounds(m.get_bounds())

m

In [12]:
cells_outline = cells.outline_to_geojson(grid)
m = folium.Map(zoom_start=12, location=[50.965, 13.922])
folium.GeoJson(cells_outline.as_geojson()).add_to(m)
m.fit_bounds(m.get_bounds())

m